# Hybrid Search in Langchain

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

API_KEY = os.getenv('PINECONE_API_KEY')

In [2]:
from langchain_community.retrievers import PineconeHybridSearchRetriever

In [3]:
from pinecone import Pinecone,ServerlessSpec
index_name = 'hybrid-search-langchain-pinecone'
## initialize the Pinecone client
pc = Pinecone(api_key=API_KEY)

# create index
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384, #dimension of dense vector
        metric='dotproduct', ## sparse values supported only for dotproduct
        spec=ServerlessSpec(cloud='aws',region='us-east-1')
    )

D:\gen-ai\langcain-tut\venv\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [4]:
index = pc.Index(index_name)
index

In [5]:
## vector embedding and sparse matrix
from langchain_community.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name = 'all-MiniLM-L6-v2')
embeddings

D:\gen-ai\langcain-tut\venv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [6]:
from pinecone_text.sparse import BM25Encoder

bm25_encoder = BM25Encoder()
bm25_encoder

In [8]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Kartikeya\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


True

In [10]:
sentences = [
    "In 2023, I visited Paris",
    "In 2022, I visited New York",
    "In 2021, I visted New Orleans",
]

## tfidf values on these sentences
bm25_encoder.fit(sentences)

## store the values in json file
bm25_encoder.dump('bm25_values.json')

## load to your bm25_encoder object

bm25_encoder = BM25Encoder().load('bm25_values.json')


00%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<?, ?it/s]

In [23]:
retriever = PineconeHybridSearchRetriever(embeddings=embeddings,sparse_encoder=bm25_encoder,index=index)

In [24]:
retriever

PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x000001A4027BD690>, index=<pinecone.data.index.Index object at 0x000001A4287BF3D0>)

In [37]:
from pinecone_text.sparse import BM25Encoder
from langchain_community.retrievers import PineconeHybridSearchRetriever

# Your sentences
sentences = [
    "In 2023, I visited Paris", 
    "In 2022, I visited New York", 
    "In 2021, I visted New Orleans"
]

# Create and fit the BM25 encoder
bm25_encoder = BM25Encoder()
bm25_encoder.fit(sentences)

# Create the retriever
# Ensure you have your embeddings and index defined beforehand
retriever = PineconeHybridSearchRetriever(
    embeddings=embeddings,  
    sparse_encoder=bm25_encoder,  
    index=index  
)

# Add texts to the retriever
retriever.add_texts(
    texts=sentences,
)

100%|██████████| 1/1 [00:00<00:00,  2.82it/s]


In [38]:
retriever.invoke("What city did i visit last")

[Document(page_content='In 2022, I visited New York'),
 Document(page_content='In 2023, I visited Paris'),
 Document(page_content='In 2021, I visted New Orleans')]